In [ ]:
#import necessary modules
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame

In [ ]:
# underlying price (per share): S;
# strike price for the option (per share): K;
# time to maturity (years): T;
# Continuously compounding risk-free interest rate: r;
# sigma is Volatility

## define the two functions: d1 and d2 in Black-Scholes model

def d1(S, K, T, r, sigma):
    return(log(S/K) + (r + sigma**2/2)*T)/sigma*sqrt(T)
def d2(S, K, T, r, sigma):
    return d1(S, K, T, r, sigma) - sigma*sqrt(T)

In [ ]:
#call option price formula
def bs_call(S, K, T, r, sigma):
    return S*norm.cdf(d1(S, K, T, r, sigma)) - K*exp(-r*T)*norm.cdf(d2(S, K, T, r, sigma))

In [ ]:
#put option price formula
def bs_put(S, K, T, r, sigma):
    return K*exp(-r*T) - S + bs_call(S, K, T, r, sigma);

In [ ]:
#formulas of all the greeks for call option
def call_delta(S, K, T, r, sigma):
    return norm.cdf(d1(S, K, T, r, sigma))
def call_gamma(S, K, T, r, sigma):
    return norm.pdf(d1(S, K, T, r, sigma)) / (S*sigma*sqrt(T))
def call_vega(S, K, T, r, sigma):
    return 0.01*(S*norm.pdf(d1(S, K, T, r, sigma))*sqrt(T))
def call_theta(S, K, T, r, sigma):
    return 0.01*(-(S*norm.pdf(d1(S, K, T, r, sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S, K, T, r, sigma)))
def call_rho(S, K, T, r, sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S, K, T, r, sigma)))

In [ ]:
#formulas of all the greeks for put option
def put_delta(S, K, T, r, sigma):
    return -norm.cdf(-d1(S, K, T, r, sigma))
def put_gamma(S, K, T, r, sigma):
    return norm.pdf(d1(S, K, T, r, sigma))/(S*sigma*sqrt(T))
def put_vega(S, K, T, r, sigma):
    return 0.01*(S*norm.pdf(d1(S, K, T, r, sigma))*sqrt(T))
def put_theta(S, K, T, r, sigma):
    return 0.01*(-(S*norm.pdf(d1(S, K, T, r, sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S, K, T, r, sigma)))
def put_rho(S, K, T, r, sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S, K, T, r, sigma)))

In [ ]:
from datetime import datetime

#input current stock price and check if it is a number
S = input("What is the current stock price? ")
while True:
    try:
        S = float(S)
        break
    except:
        print("The current stock price has to be a NUMBER.")
        S = input("What is the current stock price? ")

#Input strike price and check if it is a number.    
K = input("What is the strike price? ")
while True:
    try:
        K = float(K)
        break
    except:
        print("The strike price has to be a NUMBER.")
        K = input("What is the strike price? ")  

#Input the expiration date and calculate the difference of days between today and the expiration date (T)   
while True:
    expiration_date = input("What is the expiration date of options? (mm-dd-yyyy) ")
    try:
        expiration_date = datetime.strptime(expiration_date, "%m-%d-%Y") #check if it is in the right format
    except ValueError as e:
        print("error: %s\nTry again," % (e,))
    else:
        break

T = (expiration_date - datetime.now()).days / 365  #compute T (time until maturity)

#Input the continuously compounding risk-free interest rate and check if a number
r = input("What is the continuously compounding risk-free interest rate in percentage(%)? ")
while True:
    try:
        r = float(r)
        break
    except:
        print("The continuously compounding risk-free interest rate has to be a NUMBER.")
        r = input("What is the continuously compounding risk-free interest rate in percentage(%)? ")

#Input the volatility and check if it is a number.
sigma = input("What is the volatility in percentage(%)? ")
while True:
    try:
        sigma = float(sigma)
        if sigma > 100 or sigma < 0:    #check range of sigma
            print("The range of sigma has to be in [0,100].") 
            sigma = input("What is the volatility in percentage(%)? ")
        else:
            break
    except:
        print("The volatility has to be a NUMBER.")
        sigma = input("What is the volatility in percentage(%)? ")  

In [ ]:
#dataframe of the inputs

data = {'Symbol': ['S', 'K', 'T', 'r', 'sigma'],
        'Input': [100.0, 100.0, 1.0, 5.0, 20.0]}
input_frame = pd.DataFrame(data, columns=['Symbol', 'Input'])
input_frame.index = ['Underlying price', 'Strike price', 'Time to maturity', 'Risk-free interest rate', 'Volatility']
print(input_frame)

                        Symbol  Input
Underlying price             S  100.0
Strike price                 K  100.0
Time to maturity             T    1.0
Risk-free interest rate      r    5.0
Volatility               sigma   20.0


In [ ]:
# Black-Scholes Option Pricing Functions

#call option price
#we're writing down the formulas and deriving again instead of using the previous defined functions since the greeks require derivatives to be computed.
def black_scholes_call(S, K, T, r, sigma):
    """Calculate Black-Scholes call option price"""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    call_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    return call_price

#put option price
def black_scholes_put(S, K, T, r, sigma):
    """Calculate Black-Scholes put option price"""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    return put_price

# Greeks calculation functions for both call and put options
def calculate_greeks(S, K, T, r, sigma):
    """Calculate all Greeks for both call and put options"""
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    # call option greeks
    call_delta = norm.cdf(d1)
    call_gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    call_vega = S * norm.pdf(d1) * np.sqrt(T) / 100  # Vega per 1% change in volatility
    call_theta = -(S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + 
                   r * K * np.exp(-r*T) * norm.cdf(d2)) / 365  # Theta per day
    call_rho = K * T * np.exp(-r*T) * norm.cdf(d2) / 100  # Rho per 1% change in interest rate
    
    # put option greeks
    put_delta = call_delta - 1
    put_gamma = call_gamma  # Gamma is the same for calls and puts
    put_vega = call_vega   # Vega is the same for calls and puts
    put_theta = -(S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - 
                  r * K * np.exp(-r*T) * norm.cdf(-d2)) / 365  # Theta per day
    put_rho = -K * T * np.exp(-r*T) * norm.cdf(-d2) / 100  # Rho per 1% change in interest rate
    
    return {
        'call_delta': call_delta, 'call_gamma': call_gamma, 'call_vega': call_vega,
        'call_theta': call_theta, 'call_rho': call_rho,
        'put_delta': put_delta, 'put_gamma': put_gamma, 'put_vega': put_vega,
        'put_theta': put_theta, 'put_rho': put_rho
    }

# extract parameters values from DataFrame
S = input_frame.loc['Underlying price', 'Input']
K = input_frame.loc['Strike price', 'Input']
T = input_frame.loc['Time to maturity', 'Input']
r = input_frame.loc['Risk-free interest rate', 'Input'] / 100  # Convert percentage to decimal
sigma = input_frame.loc['Volatility', 'Input'] / 100  # Convert percentage to decimal

# compute call and put option prices
call_price = black_scholes_call(S, K, T, r, sigma)
put_price = black_scholes_put(S, K, T, r, sigma)

# compute greeks
greeks = calculate_greeks(S, K, T, r, sigma)

# results DataFrame
price_and_greeks = pd.DataFrame({
    'Call': [call_price, greeks['call_delta'], greeks['call_gamma'], 
             greeks['call_vega'], greeks['call_rho'], greeks['call_theta']],
    'Put': [put_price, greeks['put_delta'], greeks['put_gamma'], 
            greeks['put_vega'], greeks['put_rho'], greeks['put_theta']]
}, index=['Price', 'delta', 'gamma', 'vega', 'rho', 'theta'])

print("Option Prices and Greeks:")
print(price_and_greeks)

Option Prices and Greeks:
            Call       Put
Price  10.450584  5.573526
delta   0.636831 -0.363169
gamma   0.018762  0.018762
vega    0.375240  0.375240
rho     0.532325 -0.418905
theta  -0.017573 -0.004542


In [ ]:
# user input for option type and price
option = input("Put or Call option? (P/C)  ")
while option != 'P' and option != 'C':
        print("Error: This option does not match the format (P/C) \nTry again: ")
        option = input("Put or Call option? (P/C)  ")

price = input("What is the option price? ")
while True:
    try:
        Price = float(Price)
        break
    except:
        print("The option price has to be a number.")
        Price = input("What is the option price? ")

Put or Call option? (P/C)   C
What is the option price?  10.450584


The option price has to be a number.


What is the option price?  10.450584


In [ ]:
# Implied Volatility Calculator
def implied_volatility(price, S, K, T, r, option_type='C'):
    # Convert price to float if it's a string
    price = float(price)
    
    sigma = 0.001
    print(np.array([['price', 'S',  'K', 'T', 'r'], [price, S, K, T, r]]))
    
    #brute force incremental root search 
    #start with a small value of sigma and compute the c=bs option price and compare with the market price
    #this process continues by incrementing sigma by 0.001 till it reaches1
    if option_type == 'C':  
        while sigma < 1:
            Price_implied = S*norm.cdf(d1(S,K,T,r,sigma)) - K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
            if abs(price - Price_implied) < 0.001:  
                return sigma
            sigma += 0.001
        return "Could not find the right volatility of the call option."
    else:
        while sigma < 1:
            Price_implied = K*exp(-r*T) - S + bs_call(S,K,T,r,sigma)
            if abs(price - Price_implied) < 0.001:  
                return sigma
            sigma += 0.001
        return "Could not find the right volatility of the put option."  
    
# Call the function
result = implied_volatility(price, S, K, T, r)
print("the implied volatility is " + str(100 * result) + "%.")

[['price' 'S' 'K' 'T' 'r']
 ['10.450584' '100.0' '100.0' '1.0' '0.05']]
the implied volatility is 20.000000000000014%.
